<a href="https://colab.research.google.com/github/Ihsan1331/NLP_Course/blob/main/Intro_to_NLP_Ch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Prerequisites
---
link : https://huggingface.co/learn/nlp-course/chapter3/1?fw=pt

In [26]:
!pip install datasets -q
!pip install transformers[torch] -q
!pip install accelerate -U -q
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


# 1. Introduction
---
Pada Chapter ini akan membahas
1. gimana caranya menyiapkan dataset dari hub
2. caranya menggunakan high level API Trainer buat fune tuning model
3. Caranya bikin custom training loop
4. Caranya memanfaatkan 🤗 Accelerate library untuk merunning custom training loop

# 2. Preprocessing data

In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Sama seperti sebelumnya
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# Cara untuk training dalam 1 batch
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


dataset yang akan digunakan MRPC (Microsoft Research Paraphrase Corpus) dataset.  Dataset terdiri dari 5,801 pasangan kalimat, dengan label yang mengindikasikan kalau mereka hasil parafrase atau bukan.

## 2.1 Loading dataset dari Hub

---
1. Hub tidak hanya menyimpan model tapi juga menyimpan dataset
2. Saat ini pakai MRPC (Microsoft Research Paraphrase Corpus) dataset.

In [3]:
from datasets import load_dataset

# load dataset dari Hub
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
#Ambil salah satu contoh
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [5]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

## 2.2 Preprocessing dataset

In [6]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokens_sentence_1 = tokenizer(raw_train_dataset['sentence1'])
tokens_sentence_2 = tokenizer(raw_train_dataset['sentence2'])

Tapi kita tidak bisa menginput langsung 2 teks ke model dan mendapat prediksi bahwa teks tersebut hasil parafrase atau bukan. Diperlukan preprocessing yang berbeda untuk mengangani 2 teks sebagai pair. **tokenizer dapat menerima pasangan dari teks dan menyiapkannya sesuai dengan model BERT yang digunakan**


In [7]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Caranya adalah dengan menggunakan token_type_ids, token_type_ids yang memberikan informasi kepada model untuk mengetahui token mana yang merupakan teks 1 dan token mana yang merupakan teks 2.

In [8]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))

['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


Hal ini terlihat dengan adanya token spesial diantara 2 teks yang berbeda yaitu [SEP] yang memisahkan 2 teks

NB: **Tidak semua checkpoint ada token_type_ids**. token_type_ids hanya akan muncul jika saat pretrained sudah dilatih dengan token_type_ids.

In [9]:
tokenized_dataset = tokenizer(raw_train_dataset['sentence1'],
                              raw_train_dataset['sentence2'],
                              padding = True,
                              truncation = True)

Kekurangan dari metode ini adalah RAM yang diperlukan dikarenakan mereturn dictionary, sementara HF dataset disimpan dalam bentuk dataset (Apache Arrow) yang hanya memerlukan sampel disimpan dalam memory.

untuk tetap menyimpan dalam bentuk dataset gunakan Dataset.map()

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Padding tidak dimasukan karena padding semua sampel ke panjang maksimu tidak efisien. Lebih baik padding sesuai dengan panjang teks terbesar pada batch tersebut sehingga mempercepat proses training dan mengurangi bebas komputasi.

## 2.3 Dynamic Padding

Fungsi yang menyatukan sampel didalam batch dinamakan collate function. Karena kita ingin panjang dari tiap teks tidak sama (sesuai dengan maksimum dari batch tersebut) maka tidak bisa menggunakan collate function.

Akan menggunakan DataCollatorWithPadding

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Sebagai contoh ambil sampel ayitu 8 teks pertama.

In [12]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]] # Hanya ambil inupt_ids

[50, 59, 47, 67, 59, 50, 62, 32]

Pada sampel ini paling panjang adalah 67. Dengan dynamic pading khusus untuk batch ini dipad sehingga panjang tiap teks 67.

In [13]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

# 3. Fine-tuning a model with the Trainer API

In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#load dataset
raw_datasets = load_dataset("glue", "mrpc")

#pilih checkpoint
checkpoint = "bert-base-uncased"
#amil tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#tokenisasi
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

#dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

## 3.1 Training

---
1. HF punya class Trainer yang digunakan untuk fine tuning model pretrained.
2. Sebelum define Trainer perlu define class TrainingArguments yang akan menyimpan semua hyperparameter yang digunakan ntuk training dan evaluation.
3. Untuk saat ini, argument yang diperlukan hanyalah lokasi penyimpanan untuk menyimpan model dan juga checkpointnya.

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [16]:
from transformers import AutoModelForSequenceClassification
# pilih model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ada warning yang mengatakan weight (bobot) dari BERT tidak digunakan. Hal ini dikarenakan BERT belum di pretrained untuk klasifikasi, sehingga head dari pretrained model perlu dihapus dan diganti dengan head yang baru. Warning itu mengatakan bobot head masih random dan perlu di train

In [17]:
from transformers import Trainer

trainer = Trainer(
    model, #modelnya diload pakai AutoModelForSequenceClassification
    training_args, #training args dari TrainingArguments
    train_dataset=tokenized_datasets["train"], #data train dari Datasets yang udah di tokenisasi
    eval_dataset=tokenized_datasets["validation"], # data valid dari Dataset yang udah di teokenisasi
    data_collator=data_collator, # data_collator buat dynamic_padding
    tokenizer=tokenizer, #tokenizer buat tokenisasi
)

In [18]:
#training model
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.619800
1000,0.539000


TrainOutput(global_step=1377, training_loss=0.5298031520912764, metrics={'train_runtime': 219.9657, 'train_samples_per_second': 50.026, 'train_steps_per_second': 6.26, 'total_flos': 405324636337200.0, 'train_loss': 0.5298031520912764, 'epoch': 3.0})

Proses fine tunining akan dimulai dan mereport train loss tiap 500 steps. tapi tidak akan menunjukkan performa dari model. Karena:

1. Tidak menset Trainer untuk mengevaluasi saat training dengan evaluation_strategy menjadi steps atau epoch
2. Tidak memberikan Trainer compute_metrics() untuk menghitung metrik evaluasi. Sehingga hanya mengeluarkan loss saja.

## 3.2 Evaluation

In [23]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


Ouput dari model ada 2:
1. predictions.predictions -> hasil prediksi
2. predictions.label_ids -> Ground Truth

In [28]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
preds

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

Pakai argmax buat mereturn index dengan nilai tertinggi sehingga hasil dari preds adalah kelas bukan logits

In [27]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8161764705882353, 'f1': 0.8695652173913044}

Kita bisa ngitung performa dari model dengan menggunakan HF package evaluate. Disini kita menggunakan metric dari MRPC.

In [30]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Satukan semuanya. Berarti dalam 1 epoch pertama lakuin prediksi dengan bobot model di epoch tersebut -> lakuin argmax -> hitung performa dari model

In [31]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tambahkan 1 argumen di proses training yaitu compute_metrics dan juga di training_args tambahkan evaluation_strategy

In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.518138,0.776961,0.851550
2,0.611400,0.408185,0.845588,0.891938
3,0.463500,0.418831,0.850490,0.893543


TrainOutput(global_step=1377, training_loss=0.48275740768222075, metrics={'train_runtime': 196.6379, 'train_samples_per_second': 55.961, 'train_steps_per_second': 7.003, 'total_flos': 405540469624800.0, 'train_loss': 0.48275740768222075, 'epoch': 3.0})

# 4. Full Training

---
Kali ini tanpa Trainer class



In [33]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

## 4.1 Preprocessing

---
1. Hapus kolom yang tidak dapat diterima sebagai input oleh model (sentence 1 dan sentence 2)
2. ganti nama label kolom menjadi labels (model hanya bisa menerima nama labels)
3. Ubah format dataset menjadi torch

In [34]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Define dataloaders

In [35]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [36]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 75]),
 'token_type_ids': torch.Size([8, 75]),
 'attention_mask': torch.Size([8, 75])}

Tidak ada masalah pada dataloader dan masih mengaplikasikan dynamic padding

In [37]:
# Load Model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7815, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


Uji coba test dengan sample berhasil masuk ke model dan mengeluarkan hasil

In [39]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


Menentukan Optimizer dan get_scheduler

## 4.2 Training loop

In [41]:
# Gunakan GPU
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [42]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

1. tqdm buat nunjukkin progress
2. cara kerja secara umum = train-> output -> hitung loss-> backward prop -> optimizer -> update learning rate -> train
3. belum ada evaluation loop

## 4.3 Evaluation loop

In [43]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8651960784313726, 'f1': 0.9078726968174204}

## 4.4 Supercharge training loop 🤗 Accelerate
---
Training loop sebelumnya hanya bisa di 1 GPU atau TPU unutuk bisa di banyak GPU pakai 🤗 Accelerate library, dengan sedikit perubahan bsia distribusi training di >1 GPU atau TPU

In [44]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

# 5. Fine-Tuning Check

# 6. Quiz